<a href="https://colab.research.google.com/github/VFA21SCM91V/ML_projects/blob/main/LSTM%20seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
!pip install plotly

     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
import sentencepiece as sp
import time
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from numpy.random import shuffle

In [4]:
from google.colab import files
uploaded = files.upload()

Saving cmn.txt to cmn.txt


In [5]:
data_path = "cmn.txt"

In [6]:
start_time = time.time()
train_df = pd.read_csv(data_path, sep='\t', names=["EN","NL"])
print(f"File loading time: {time.time()-start_time}")
print(train_df.shape)

File loading time: 0.03905224800109863
(26388, 2)


In [7]:
train_df.head()

,EN,NL
0,Hi.,嗨 。
1,Hi.,你好 。
2,Run.,你 用 跑 的 。
3,Wait!,等等 ！
4,Wait!,等 一下 ！


In [19]:
train_df.iloc[-1, 1]

'如果 一個 人 在 成人 前 沒 有 機會習 得 目標 語言 ， 他 對 該 語言 的 認識 達 到 母語者 程度 的 機會 是 相當 小 的 。'

In [8]:
def write_trainer_file(col, filename):
    texts = list(col.values)
    with open(filename, 'w',encoding='utf-8') as f:
        for text in texts:
            f.write(text + "\n")

In [9]:
en_sp_trainer = "en_spm.txt"
write_trainer_file(train_df["EN"], en_sp_trainer)

In [10]:
nl_sp_trainer = "nl_spm.txt"
write_trainer_file(train_df["NL"], nl_sp_trainer)

In [11]:
def createSPModel(trainer_file, model_prefix, vocab_size, sp):
    spm_train_param = f"--input={trainer_file} --model_prefix={model_prefix} --vocab_size={vocab_size}"
    sp.SentencePieceTrainer.Train(spm_train_param)
    lang_sp = sp.SentencePieceProcessor()
    lang_sp.Load(f"{model_prefix}.model")
    return lang_sp

In [13]:
en_sp = createSPModel(en_sp_trainer, "en_sp", 6100, sp)

In [14]:
print(en_sp.EncodeAsPieces("This is a test."))
print(en_sp.EncodeAsIds("This is a test."))
print(en_sp.DecodeIds(en_sp.EncodeAsIds("This is a test.")))

['▁Thi', 's', '▁is', '▁a', '▁test', '.']
[79, 10, 12, 8, 748, 3]
This is a test.


In [16]:
nl_sp = createSPModel(nl_sp_trainer, "nl_sp", 6500, sp)

In [20]:
nl_sp.EncodeAsPieces("如果 一個 人 在 成人 前 沒 有 機會習 得 目標 語言 ， 他 對 該 語言 的 認識 達 到 母語者 程度 的 機會 是 相當 小 的 。")

['▁如',
 '果',
 '▁一個',
 '▁人',
 '▁在',
 '▁成',
 '人',
 '▁前',
 '▁沒',
 '▁有',
 '▁機會',
 '習',
 '▁得',
 '▁目',
 '標',
 '▁語言',
 '▁',
 ',',
 '▁他',
 '▁對',
 '▁該',
 '▁語言',
 '▁的',
 '▁認識',
 '▁達',
 '▁到',
 '▁母語',
 '者',
 '▁程',
 '度',
 '▁的',
 '▁機會',
 '▁是',
 '▁相',
 '當',
 '▁小',
 '▁的',
 '▁。']

In [21]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import optimizers
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [22]:
def encode_sentence(df, lang, spm):
    lang_pieces = []
    lang_lens = []
    for index, row in df.iterrows():
        lang_piece = spm.EncodeAsIds(row[lang])
        lang_pieces.append(lang_piece)
        lang_lens.append(len(lang_piece)) 
    df[f"{lang}_pieces"] = lang_pieces
    df[f"{lang}_len"] = lang_lens

In [23]:
start_time = time.time()
encode_sentence(train_df, "EN", en_sp)
encode_sentence(train_df, "NL", nl_sp)
print(f"Encoding time: {time.time()-start_time} sec")

Encoding time: 4.119035005569458 sec


In [24]:
train_df.tail()

,EN,NL,EN_pieces,EN_len,NL_pieces,NL_len
26383,"Tom didn't know how to translate the word ""com...",汤姆 不知 如何 翻译 “ 计算机 ” 一词 ， 因为 同 他 谈话 的 人 从未见过 一台 。,"[14, 52, 19, 5, 13, 55, 185, 6, 1604, 7, 538, ...",27,"[15, 3458, 120, 466, 2129, 325, 3373, 330, 18,...",26
26384,"Even now, I occasionally think I'd like to see...",即使 是 现在 ， 我 偶尔 还是 想 见到 你 。 不是 今天 的 你 ， 而是 我 记忆...,"[995, 19, 109, 21, 4, 2037, 73, 4, 5, 44, 39, ...",32,"[1957, 1156, 12, 189, 41, 4, 16, 5, 1784, 1457...",33
26385,It's very easy to sound natural in your own na...,你 很 容易 把 母语 说 得 通顺 流畅 ， 却 很 容易 把 非 母语 说 得 不 自然 。,"[50, 5, 10, 64, 376, 6, 885, 1605, 264, 16, 34...",32,"[8, 19, 445, 413, 42, 422, 530, 52, 78, 427, 3...",28
26386,"I got fired from the company, but since I have...",虽然 我 被 公司 解雇 了 ， 但是 我 还 有点 存款 ， 所以 目前 不用 担心 生计...,"[4, 102, 1413, 84, 7, 662, 21, 136, 609, 4, 23...",33,"[4, 2280, 320, 5, 74, 770, 492, 3518, 7, 4, 16...",30
26387,If a person has not had a chance to acquire hi...,如果 一個 人 在 成人 前 沒 有 機會習 得 目標 語言 ， 他 對 該 語言 的 認識...,"[4, 166, 8, 342, 58, 51, 80, 8, 635, 6, 4359, ...",40,"[120, 114, 88, 39, 11, 203, 33, 271, 75, 17, 1...",38


In [25]:
def plotLangLen(lang1, lang2):
    trace1 = go.Histogram(
        x=train_df[f"{lang1}_len"].values,
        opacity=0.75,
        name = f"Length of {lang1} sentences",
        marker=dict(color='rgba(171, 50, 96, 0.6)'))
    trace2 = go.Histogram(
        x=train_df[f"{lang2}_len"].values,
        opacity=0.75,
        name = f"Length of {lang2} sentences",
        marker=dict(color='rgba(12, 50, 196, 0.6)'))

    data = [trace1, trace2]
    layout = go.Layout(barmode='overlay',
                       title=f"Lengths of {lang1} and {lang2} sentences",
                       xaxis=dict(title='Length'),
                       yaxis=dict( title='Count'),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, config={'showLink': True})

In [26]:
plotLangLen("EN", "NL")

In [27]:
en_vocab_size = en_sp.get_piece_size()
nl_vocab_size = nl_sp.get_piece_size()
print(f"EN vocab size: {en_vocab_size}")
print(f"NL vocab size: {nl_vocab_size}")

EN vocab size: 6100
NL vocab size: 6500


In [28]:
print(en_sp.piece_to_id('__MUST_BE_UNKNOWN__'))
print(en_sp.id_to_piece(0))

0
<unk>


In [29]:
en_max_length = train_df["EN_len"].max()
nl_max_length = train_df["NL_len"].max()
#30 for faster processing time
en_max_length=30
nl_max_length=en_max_length
print(en_max_length)
print(nl_max_length)

30
30


In [30]:
#use post padding to fill up short sentence with 0
en_padded_seq = pad_sequences(train_df["EN_pieces"].tolist(), maxlen=en_max_length, padding='post')
nl_padded_seq = pad_sequences(train_df["NL_pieces"].tolist(), maxlen=nl_max_length, padding='post')

In [31]:
#pick a sample
en_padded_seq[2]

array([2884,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [32]:
train_seq_df = pd.DataFrame( {'en_seq':en_padded_seq.tolist(), 'nl_seq':nl_padded_seq.tolist()})

In [33]:
train_seq_df.tail()

,en_seq,nl_seq
26383,"[14, 52, 19, 5, 13, 55, 185, 6, 1604, 7, 538, ...","[15, 3458, 120, 466, 2129, 325, 3373, 330, 18,..."
26384,"[109, 21, 4, 2037, 73, 4, 5, 44, 39, 6, 113, 9...","[189, 41, 4, 16, 5, 1784, 1457, 146, 44, 29, 4..."
26385,"[10, 64, 376, 6, 885, 1605, 264, 16, 34, 486, ...","[8, 19, 445, 413, 42, 422, 530, 52, 78, 427, 3..."
26386,"[84, 7, 662, 21, 136, 609, 4, 23, 8, 204, 133,...","[4, 2280, 320, 5, 74, 770, 492, 3518, 7, 4, 16..."
26387,"[4359, 45, 2831, 537, 85, 7, 67, 46, 5, 10, 72...","[75, 17, 1179, 684, 78, 1664, 2754, 1036, 4, 1..."


In [34]:
def define_model(input_vocab,output_vocab, input_length,output_length,output_dim):
      model = Sequential()
      #mark_zero , set 0 as special character reserved for unknown words  
      model.add(Embedding(input_vocab, output_dim, input_length=input_length, mask_zero=True))
      model.add(LSTM(output_dim))
      #repeat the input (n) times
#return_sequences=True returns all the outputs the encoder observed in the past, while RepeatVector repeats the very last output of the encoder.    
      model.add(RepeatVector(output_length))
    #return the full sequences
      model.add(LSTM(output_dim, return_sequences=True))
      #model.add(TimeDistributed(Dense(output_vocab, activation='softmax')))
      
      model.add(Dense(output_vocab, activation='softmax'))
      return model

In [35]:
train_seq_df.shape

(26388, 2)

In [36]:
train, test = train_test_split(train_seq_df, test_size=0.1, random_state = 3)

In [37]:
trainX = np.asarray(train["nl_seq"].tolist())
trainY = np.asarray(train["en_seq"].tolist())

testX = np.asarray(test["nl_seq"].tolist())
testY = np.asarray(test["en_seq"].tolist())

In [38]:
model = define_model(nl_vocab_size, en_vocab_size, nl_max_length, en_max_length, 1024)

In [39]:
#RMSprop is recommended for RNN, sparse_categorical_crossentropy for densed target output as integers
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [40]:
def encode_output(sequences, vocab_size):
   ylist = list()
   for sequence in sequences:
    encoded = to_categorical(sequence, num_classes=vocab_size)
    ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [41]:
filename = 'nmt_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=15, batch_size=64, validation_split = 0.1,callbacks=[checkpoint], 
                    verbose=1)

Epoch 1/15
334/334 [==============================] - ETA: 0s - loss: 1.8158
Epoch 00001: val_loss improved from inf to 1.60443, saving model to nmt_model


INFO:tensorflow:Assets written to: nmt_model/assets


INFO:tensorflow:Assets written to: nmt_model/assets


334/334 [==============================] - 44s 110ms/step - loss: 1.8158 - val_loss: 1.6044
Epoch 2/15
333/334 [============================>.] - ETA: 0s - loss: 1.4837
Epoch 00002: val_loss improved from 1.60443 to 1.45099, saving model to nmt_model


INFO:tensorflow:Assets written to: nmt_model/assets


...

INFO:tensorflow:Assets written to: nmt_model/assets


334/334 [==============================] - 35s 105ms/step - loss: 1.0263 - val_loss: 1.2346
Epoch 8/15
334/334 [==============================] - ETA: 0s - loss: 0.9482
Epoch 00008: val_loss improved from 1.23458 to 1.22957, saving model to nmt_model


INFO:tensorflow:Assets written to: nmt_model/assets


INFO:tensorflow:Assets written to: nmt_model/assets


334/334 [==============================] - 35s 105ms/step - loss: 0.9482 - val_loss: 1.2296
Epoch 9/15
334/334 [==============================] - ETA: 0s - loss: 0.8675
Epoch 00009: val_loss did not improve from 1.22957
334/334 [==============================] - 23s 69ms/step - loss: 0.8675 - val_loss: 1.2337
Epoch 10/15
334/334 [==============================] - ETA: 0s - loss: 0.7904
Epoch 00010: val_loss did not improve from 1.22957
334/334 [==============================] - 23s 70ms/step - loss: 0.7904 - val_loss: 1.2519
Epoch 11/15
333/334 [============================>.] - ETA: 0s - loss: 0.7164
Epoch 00011: val_loss did not improve from 1.22957
334/334 [==============================] - 23s 69ms/step - loss: 0.7162 - val_loss: 1.2663
Epoch 12/15
334/334 [==============================] - ETA: 0s - loss: 0.6410
Epoch 00012: val_loss did not improve from 1.22957
334/334 [==============================] - 23s 68ms/step - loss: 0.6410 - val_loss: 1.2957
Epoch 13/15
333/334 [========

In [42]:
trace1 = go.Scatter(
    y=history.history['loss'],
    name = "Training Loss",
    marker=dict(color='rgba(171, 50, 96, 0.6)'))
trace2 = go.Scatter(
    y=history.history['val_loss'],
    name = "Validation Loss",
    marker=dict(color='rgba(12, 50, 196, 0.6)'))

data = [trace1, trace2]
layout = go.Layout(title='Loss and Val_Loss in 15 Epochs',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict( title='Loss'),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, config={'showLink': True})

In [43]:
model = load_model('nmt_model')

In [44]:
preds = np.argmax(model.predict(testX.reshape((testX.shape[0],testX.shape[1]))), axis=-1)

In [45]:
def get_word(ids, tokenizer):
    return tokenizer.DecodeIds(list(filter(lambda a: a != 0, ids.tolist())))

In [46]:
import random

In [47]:
y_indexs = random.sample(range(len(testY)),  30)

for y_index in y_indexs: 
  print(f"--- Index: {y_index}")
  print(f"NL: {get_word(testX[y_index], nl_sp)}") 
  print(f"EN: {get_word(testY[y_index], en_sp)}") 
  print(f"MT: {get_word(preds[y_index], en_sp)}")
 # print(get_word(k_preds[0][y_index], en_sp))
 # print(get_word(k_preds[1][y_index], en_sp))
 # print(get_word(k_preds[2][y_index], en_sp))

--- Index: 2430
NL: 我 讓 我 的 哥哥 修理 我 的 腳踏車 。
EN: I had my brother repair my bicycle.
MT: I've brother my my my my..
--- Index: 1600
NL: 全家 都 病倒 了 。
EN: The whole family was sick in bed.
MT: The train was coming..
--- Index: 2261
NL: 甲 之 蜜糖 , 乙 之 砒霜 。
EN: One man's meat is another man's poison.
MT: An,,,,, the the..
--- Index: 1183
NL: 跟 你 一起 吃午 飯會 讓 我 高興 。
EN: Eating lunch with you would make me happy.
MT: Your you help me me me..
--- Index: 1773
NL: 你 也 是 学生 不是 吗 ?
EN: You're a student, too, aren't you?
MT: Younnnt you you you?
--- Index: 180
NL: 把 我 的 啤酒 給我 。
EN: Give me my beer.
MT: Bring me me me salt.
--- Index: 2397
NL: 他们 每天 必须 工作 8 小时 。
EN: They must work  hours a day.
MT: They work to a a last..
--- Index: 408
NL: 汤姆 希望 玛丽 不会 跟 约翰 谈恋爱 。
EN: Tom didn't like the idea of Mary going out with John.
MT: Tom told Mary''' would to to to..
--- Index: 1941
NL: 我們 要 等等 看會 發生 甚麼 。
EN: We'll wait and see what happens.
MT: We'll do to what what we do..
--- Index: 2385
NL: 我 没法 改变 昨天 发生 的 事情 